In [ ]:
# import the necessary packages for processing
import anndata
import pysam
import numpy as np
import os


: 

Load the necessary dataset with anndata

Download the fasta file from here: https://hgdownload.soe.ucsc.edu/goldenPath/hg19/

Remember different datasets may have different reference genomes, so it would be better to check at the scATAC-seq dataset decription before you start

In [ ]:
adata = anndata.read_h5ad("../data/buen18.h5ad")
fasta_file = 'hg19.fa'
fasta_open = pysam.Fastafile(fasta_file)


Extract the peak sequences and save to txt file

In [ ]:
chrom = np.array(adata.var['chr'])
start = np.array(adata.var['start'])
end = np.array(adata.var['end'])
seq_dna = []
for i in range(len(chrom)):
    seq_dna.append(fasta_open.fetch(chrom[i], int(start[i]), int(end[i])).upper())

Download the DNABERT pre-trained model from https://drive.google.com/file/d/1BJjqb5Dl2lNMg2warsFQ0-Xvn1xxfFXC/view?usp=sharing

Unzip the DNABERT pre-trained model

Use transformers to tokenize DNA peak sequences to tokens and save to h5 file

In [ ]:
import h5py
import anndata
from transformers import AutoTokenizer, AutoModel
import torch 
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("../gfm_checkpoint/6-new-12w-0")
kmer = 6
count = 0
device = "cuda"


token_array = []
for seq in seq_dna:
    if len(seq) > 512:
        gap = int((len(seq) - 512) //2) + 1
        seq = seq[gap:-gap]
    processed = ""
    for q in range(len(seq) - kmer):
        processed +=  seq[q:q+kmer] +" "
    token_array.append(tokenizer.encode_plus(processed,padding="max_length")['input_ids'])

token_array = np.array(token_array)
adata = anndata.AnnData(token_array)
adata.write("../data/buen18_seq.h5")